In [1]:
import pyspark.sql.functions as f
import pyspark.sql.types
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    ArrayType,
    TimestampType,
)
from pyspark.sql.session import SparkSession
import os
spark = SparkSession.builder.getOrCreate()

21/08/11 20:05:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [11]:
test_schema = StructType(
    [
        StructField("SM_CLIENTIP", StringType()),
        StructField("SM_AGENTNAME", StringType()),
        StructField("SM_TIMESTAMP", StringType()),
    ]
)

test_1_data = [
    (
        "User_A",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/63.0.3239.84",
        "2018-01-01T18:32:26.000+0000",
    ),
    (
        "User_A",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/63.0.3239.84 Safari/537.36",
        "2018-01-01T18:32:26.000+0000",
    ),
    (
        "User_A",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/63.0.3239.84",
        "2018-01-01T18:32:26.000+0000",
    ),
    (
        "User_A",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/63.0.3239.84 Safari/537.36",
        "2018-01-01T18:32:26.000+0000",
    ),
]

df = spark.createDataFrame(test_1_data, schema=test_schema)
df.show()

+-----------+--------------------+--------------------+
|SM_CLIENTIP|        SM_AGENTNAME|        SM_TIMESTAMP|
+-----------+--------------------+--------------------+
|     User_A|Mozilla/5.0 (Maci...|2018-01-01T18:32:...|
|     User_A|Mozilla/5.0 (Maci...|2018-01-01T18:32:...|
|     User_A|Mozilla/5.0 (Maci...|2018-01-01T18:32:...|
|     User_A|Mozilla/5.0 (Maci...|2018-01-01T18:32:...|
+-----------+--------------------+--------------------+



In [13]:
ans_1_schema = StructType(
    [
        StructField("SM_CLIENTIP", StringType()),
        StructField(
            "windowtmp",
            StructType(
                [
                    StructField("start", StringType(),False),
                    StructField("end", StringType()),
                ]
            ),
            False,
        ),
        StructField("Parsed_Agent_String", StringType()),
    ]
)


ans_1_data = [
    (
        "User_A",
        {
            "start": "2018-01-01T18:30:00.000+0000",
            "end": "2018-01-01T18:45:00.000+0000",
        },
        [
            {
                "platform": {"name": "Mac OS", "version": "X 10.12.6"},
                "os": {"name": "Macintosh"},
                "bot": False,
                "flavor": {"name": "MacOS", "version": "X 10.12.6"},
                "browser": {"name": "Chrome", "version": "63.0.3239.84"},
            }
        ],
    )
]

ans_1_df = spark.createDataFrame(ans_1_data, schema=ans_1_schema)
ans_1_df = ans_1_df.withColumn(
            "window",
            f.col("windowtmp").cast(
                StructType(
                    [
                        StructField("start", TimestampType()),
                        StructField("end", TimestampType()),
                    ]
                )
            ),
        ).drop("windowtmp")
ans_1_df = ans_1_df.select(["SM_CLIENTIP", "window", "Parsed_Agent_String"])
ans_1_df.show()

+-----------+--------------------+--------------------+
|SM_CLIENTIP|              window| Parsed_Agent_String|
+-----------+--------------------+--------------------+
|     User_A|{2018-01-01 18:30...|[{flavor={name=Ma...|
+-----------+--------------------+--------------------+



In [15]:
ans_2_schema = StructType(
    [
        StructField("SM_CLIENTIP", StringType()),
        StructField(
            "windowtmp",
            StructType(
                [
                    StructField("start", StringType()),
                    StructField("end", StringType()),
                ]
            ),
            False,
        ),
        StructField("SM_AGENTNAME", ArrayType(StringType(), False), False),
    ]
)

ans_2_data = [
    (
        "User_A",
        {
            "start": "2018-01-01T18:30:00.000+0000",
            "end": "2018-01-01T18:45:00.000+0000",
        },
        [
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/63.0.3239.84",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/63.0.3239.84 Safari/537.36",
        ],
    )
]

ans_2_df = spark.createDataFrame(ans_2_data, schema=ans_2_schema)
ans_2_df = ans_2_df.withColumn(
            "window",
            f.col("windowtmp").cast(
                StructType(
                    [
                        StructField("start", TimestampType()),
                        StructField("end", TimestampType()),
                    ]
                )
            ),
        ).drop("windowtmp")

ans_2_df = ans_2_df.select(["SM_CLIENTIP", "window", "SM_AGENTNAME"])
ans_2_df.show()

+-----------+--------------------+--------------------+
|SM_CLIENTIP|              window|        SM_AGENTNAME|
+-----------+--------------------+--------------------+
|     User_A|{2018-01-01 18:30...|[Mozilla/5.0 (Mac...|
+-----------+--------------------+--------------------+

